# Pre-Work

Before you continue, run the below query to get the schema you have access to. You will need this to update the queries below.

In [ ]:
-- Get Schema Name
SELECT REPLACE(REPLACE(name, ' ', ''), 'RUBRIKDEMO\', '') as 'Schema Name'
FROM sys.server_principals WHERE principal_id > 10

# Live Mount Demo

## Look at the environment before the Live Mount

In this section we want to look at the current list of databases and the size of each database. This way we can understand what is out there currently before we make a change and create a Live Mount. Looking at the left hand side of your screen you will see the SQL Server you are connect to and the list of databases under the database folder. The below shows you the database its size, which you cannot easily get from the left hand side.

In [ ]:
-- List database and its size
WITH fs
AS
(
    SELECT database_id, type, size * 8.0 / 1024 size
    FROM sys.master_files
)
SELECT name
    , (SELECT SUM(size) FROM fs WHERE type = 0 AND fs.database_id = db.database_id) DataFileSizeInMB
    , (SELECT SUM(size) FROM fs WHERE type = 1 AND fs.database_id = db.database_id) LogFileSizeInMB
FROM sys.databases db
WHERE db.database_id > 4
ORDER BY name

# Do a bad thing

## Delete without a WHERE clause

A user is doing some work in the database. They need to do some clean up and they want to delete 1 entry from a table. However instead they made a mistake and left their WHERE clause commented out and mistakenly deleted all records from the table.

In [ ]:
-- Pick one of the below tables and uncomment out the line by removing the -- at the beginning of line 6, 10 or 14
-- Update the schema name to be the value you received in the Pre-Work section. 
-- Update the table name to have 01-05 at the end. 

-- DC Characters
-- DELETE FROM [GaiaSharedAccountGroup01].[DC_Characters_01] 
-- WHERE name = 'thomas wayne (new earth)'

-- Marvel Characters
-- DELETE FROM [GaiaSharedAccountGroup01].[Marvel_Characters_01] 
-- WHERE name = 'charles xavier (earth-616)'

-- Netflix Titles
-- DELETE FROM [GaiaSharedAccountGroup01].[Netflix_Titles_01] 
-- WHERE title = 'GoodFellas'


## Live Mount a Database

A databaes of any size can be Live Mounted, but for this demo, take the largest database, which will either be named Cork or PaloAlto.

  

To create the Live Mount, you will need to log into **Rubrik Security Cloud** (RSC). You can follow these steps to create your Live Mount. 

[Rubrik Recovery Option 3: Live Mount](https://rubrik.atlassian.net/wiki/spaces/SEK/pages/2472444754/Recovery+Option+3%3A+Live+Mount)

## Show the Location of the Database Files

The below section shows where all of the files are located for each database. As you can see, "normal" databases are located on the C drive on mount points. A Live Mount database is located on a SMB3 share located on the Rubrik storage. 

  

A Live Mounted database is a fully functional database. Any read or write operation that you can do against any other database on the SQL Server can be done against one that is Live Mounted. The only difference between a Live Mounted database and a "normal" database is the storage it sits on. A "normal" database sits on your directly attached storage allocated from your SAN, while a Live Mounted database is on Rubrik storage attached via a secure SMB3 share.

In [ ]:
-- Show the Location of the Database Files
SELECT DB_NAME(database_id) as 'Database Name'
    , type_desc as 'File Type'
    , name as 'Logical Name'
    , physical_name as 'Physical Name'
    , state_desc
    , size * 8.0 / 1024 as 'File in MB'
FROM sys.master_files
WHERE database_id > 4
ORDER BY database_id

## Put the records back

In [ ]:
-- Pick one of the below tables and uncomment out the line by removing the -- at the beginning of the line

-- DC Characters
-- INSERT INTO [GaiaSharedAccountGroup01].[DC_Characters_01]
-- SELECT * FROM [LIVE MOUNT DATABASE].[GaiaSharedAccountGroup02].[DC_Characters_01]
-- WHERE name = 'thomas wayne (new earth)'

-- Marvel Characters
-- INSERT INTO [GaiaSharedAccountGroup01].[Marvel_Characters_01]
-- SELECT * FROM [LIVE MOUNT DATABASE].[GaiaSharedAccountGroup01].[Marvel_Characters_01]
-- WHERE name = 'charles xavier (earth-616)'

-- Netflix Titles
-- INSERT INTO [GaiaSharedAccountGroup01].[Netflix_Titles_01]
-- SELECT * FROM [LIVE MOUNT DATABASE].[GaiaSharedAccountGroup02].[Netflix_Titles_01]
-- WHERE title = 'GoodFellas'

## Confirm the records are back in the table

In [ ]:
-- Pick one of the below tables and uncomment out the line by removing the -- at the beginning of the line

-- DC Characters
SELECT * FROM [GaiaSharedAccountGroup02].[DC_Characters_01]
WHERE name = 'thomas wayne (new earth)'


-- Marvel Characters
SELECT * FROM [GaiaSharedAccountGroup01].[Marvel_Characters_01]
WHERE name = 'charles xavier (earth-616)'

-- Netflix Titles
SELECT * FROM [GaiaSharedAccountGroup02].[Netflix_Titles_01]
WHERE title = 'GoodFellas'

# Clean Up

Now that the records are back in the table, we can get rid of the Live Mounted database. Go back into RSC and unount the database you mounted.

## Confirm that the Live Mount is not on the SQL Server

In [ ]:
-- List database and its size
WITH fs
AS
(
    SELECT database_id, type, size * 8.0 / 1024 size
    FROM sys.master_files
)
SELECT name
    , (SELECT SUM(size) FROM fs WHERE type = 0 AND fs.database_id = db.database_id) DataFileSizeInMB
    , (SELECT SUM(size) FROM fs WHERE type = 1 AND fs.database_id = db.database_id) LogFileSizeInMB
FROM sys.databases db
WHERE db.database_id > 4
ORDER BY name